In [1]:
import torch
print(torch.__version__)

1.9.0.dev20210520+cu102


In [2]:
torch.cuda.is_available

torch.cuda.device_count()

torch.cuda.current_device()

<function torch.cuda.is_available() -> bool>

In [ ]:
#############################################################################

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [7]:
# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:

class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

In [9]:

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size), #(100, 5)
                         batch_size=batch_size, shuffle=True)

In [10]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

In [13]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

Let's use 3 GPUs!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [14]:

for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size	In Model: input size  torch.Size([10, 5]) output size torch.Size([10, 5])torch.Size([10, 2])
	In Model: input size output size torch.Size([10, 2])
 torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
 	In Model: input size torch.Size([10, 5])torch.Size([10, 5]) output size torch.Size([10, 2])
 output size torch.Size([10, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size	In Model: input size  	In Model: input sizetorch.Size([4, 5]) output size torch.Size([4, 2])
tor